Install qlatent is required

In [2]:
!pip install qlatent

In [ ]:
import unittest
import importlib
from tqdm.auto import tqdm
from qlatent.qmnli.qmnli import *
from qlatent.qmnli.qmnli import _QMNLI, QMNLI

import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

In [ ]:
import qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli
from qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli import *
import qpsychometric.personality_traits.big5.big5_qmnli
from qpsychometric.personality_traits.big5.big5_qmnli import *
import qpsychometric.personality_traits.short_dark_triad.sd3_qmnli
from qpsychometric.personality_traits.short_dark_triad.sd3_qmnli import *
import qpsychometric.mental_health.sense_of_coherence.soc_qmnli
from qpsychometric.mental_health.sense_of_coherence.soc_qmnli import *
import qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli
from qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli import *
import qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli
from qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli import *
import qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli
from qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli import *

In [88]:
def test_questionnaire(module, questionnaire_qmnli : list[QMNLI]) -> list[bool]:
    
    questions_equalities = []

    for Q in tqdm(questionnaire_qmnli):
        Qs = split_question(Q,
                            index=Q.index,
                            scales=[Q.scale],
                            softmax=[True],
                            filters={'unfiltered':{},
                                    "positiveonly":Q().get_filter_for_postive_keywords()
                                    },
                            )
        print(Qs[0]._descriptor['Ordinal'])
        Qs[0].run(mnli)
        q_pdf = Qs[0]._pdf
        Q_same = Q.__name__
        Q_func = getattr(module, Q_same)
        Qs = split_question(Q_func,
                            index=Q_func.index,
                            scales=[Q_func.scale],
                            softmax=[True],
                            filters={'unfiltered':{},
                                    "positiveonly":Q().get_filter_for_postive_keywords()
                                    },
                            )
        print(Qs[0]._descriptor['Ordinal'])
        Qs[0].run(mnli)
        questions_equalities.append(q_pdf.equals(Qs[0]._pdf))
    
    return questions_equalities

def get_middle_question(questionnaire):
    middle = len(questionnaire) // 2
    Q = questionnaire[middle]
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    return Qs[0]

class TestQuestionnairesMethods(unittest.TestCase):

    asi_module = qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli
    asi_questionnaire = asi_qmnli
    
    big5_module = qpsychometric.personality_traits.big5.big5_qmnli
    big5_questionnaire = big5_qmnli

    sd3_module = qpsychometric.personality_traits.short_dark_triad.sd3_qmnli
    sd3_questionnaire = sd3_qmnli

    soc_module = qpsychometric.mental_health.sense_of_coherence.soc_qmnli
    soc_questionnaire = soc_qmnli

    phq_module = qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli
    phq_questionnaire = phq_qmnli

    gad_module = qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli
    gad_questionnaire = gad_qmnli

    compassion_scale_module = qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli
    compassion_scale_questionnaire = compassion_scale_qmnli
    
    def test_asi_questionnaire(self):
        questions_equalities = test_questionnaire(self.asi_module, self.asi_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_big5_questionnaire(self):
        questions_equalities = test_questionnaire(self.big5_module, self.big5_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_sd3_questionnaire(self):
        questions_equalities = test_questionnaire(self.sd3_module, self.sd3_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_soc_questionnaire(self):
        questions_equalities = test_questionnaire(self.soc_module, self.soc_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_phq_questionnaire(self):
        questions_equalities = test_questionnaire(self.phq_module, self.phq_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_gad_questionnaire(self):
        questions_equalities = test_questionnaire(self.gad_module, self.gad_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_compassion_scale_questionnaire(self):
        questions_equalities = test_questionnaire(self.compassion_scale_module, self.compassion_scale_questionnaire)
        self.assertTrue(expr=False not in questions_equalities)

    def test_asi_middle_question(self):
        kw_fairness = ['sexist', 'prejudiced', 'improper',]
        kw_favors   = ['legitimate', 'rightful', 'reasonable']
        kw_all      = kw_fairness + kw_favors
        kw_all.sort()
        
        middle_question = get_middle_question(self.asi_questionnaire)
        kw = list(middle_question._dimensions['index'].keys())
        kw.sort()

        self.assertEqual(kw_all, kw)

    def test_big5_middle_question(self):
        emo_pos = ['sociable', 'in the center of attention']
        emo_neg = ['quiet', 'reserved', 'shy']
        emo_all = emo_pos + emo_neg
        emo_all.sort()
        
        middle_question = get_middle_question(self.big5_questionnaire)
        emo = list(middle_question._dimensions['emotion'].keys())
        emo.sort()

        self.assertEqual(emo_all, emo)

    def test_sd3_middle_question(self):
        emo_pos = ['like', 'love', 'want to']
        emo_neg = ['hate', 'dislike', 'despise']
        emo_all = emo_pos + emo_neg
        emo_all.sort()
        
        middle_question = get_middle_question(self.sd3_questionnaire)
        emo = list(middle_question._dimensions['emotion'].keys())
        emo.sort()

        self.assertEqual(emo_all, emo)

    def test_soc_middle_question(self):
        kw_attitude_pos = ['clear', 'coherent', 'logical', 'comprehensible']
        kw_attitude_neg   = ['mixed-up', 'confounded']
        kw_all      = kw_attitude_pos + kw_attitude_neg
        kw_all.sort()
        
        middle_question = get_middle_question(self.soc_questionnaire)
        kw = list(middle_question._dimensions['index'].keys())
        kw.sort()

        self.assertEqual(kw_all, kw)

    def test_phq_middle_question(self):
        emo_pos = ['poor appetite', 'been overeating']
        emo_neg = ['healthy appetite', 'satisfying appetite']
        emo_all = emo_pos + emo_neg
        emo_all.sort()
        
        middle_question = get_middle_question(self.phq_questionnaire)
        emo = list(middle_question._dimensions['emotion'].keys())
        emo.sort()

        self.assertEqual(emo_all, emo)

    def test_gad_middle_question(self):
        emo_pos = ['trouble', 'difficulty']
        emo_neg = ['no problem', 'an easy time']
        emo_all = emo_pos + emo_neg
        emo_all.sort()
        
        middle_question = get_middle_question(self.gad_questionnaire)
        emo = list(middle_question._dimensions['emotion'].keys())
        emo.sort()

        self.assertEqual(emo_all, emo)

    def test_compassion_scale_middle_question(self):
        kw_attitude_pos = ['correct', 'true', 'accurate', 'a fact']
        kw_attitude_neg = ['incorrect', 'false', 'imprecise', 'invalid']
        kw_all = kw_attitude_pos + kw_attitude_neg
        kw_all.sort()
        
        middle_question = get_middle_question(self.compassion_scale_questionnaire)
        kw = list(middle_question._dimensions['index'].keys())
        kw.sort()

        self.assertEqual(kw_all, kw)

def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestQuestionnairesMethods('test_asi_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_big5_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_sd3_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_soc_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_phq_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_gad_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_compassion_scale_questionnaire'))
    suite.addTest(TestQuestionnairesMethods('test_asi_middle_question'))
    suite.addTest(TestQuestionnairesMethods('test_big5_middle_question'))
    suite.addTest(TestQuestionnairesMethods('test_sd3_middle_question'))
    suite.addTest(TestQuestionnairesMethods('test_soc_middle_question'))
    suite.addTest(TestQuestionnairesMethods('test_phq_middle_question'))
    suite.addTest(TestQuestionnairesMethods('test_gad_middle_question'))
    suite.addTest(TestQuestionnairesMethods('test_compassion_scale_middle_question'))
    return suite

In [89]:
runner = unittest.TextTestRunner()
runner.run(suite())

  9%|▉         | 2/22 [00:00<00:01, 14.91it/s]

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
1
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
1
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
2
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
2
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
3
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
3
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
4


 32%|███▏      | 7/22 [00:00<00:00, 19.34it/s]

(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
4
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
5
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
5
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
6
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
6
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
7
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
7
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
8
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
8


 45%|████▌     | 10/22 [00:00<00:00, 20.18it/s]

...

 True unfiltered
(['index'], 'frequency') True positiveonly
19


..

(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
5
(['emotion'], 'intensifier') True unfiltered
(['emotion'], 'intensifier') True positiveonly
4
(['index'], 'frequency') True unfiltered
(['index'], 'frequency') True positiveonly
13


.
----------------------------------------------------------------------
Ran 14 tests in 5.611s

OK


<unittest.runner.TextTestResult run=14 errors=0 failures=0>

In [ ]:
def split_question(Q, index, scales, softmax, filters):
  result = []
  for s in scales:
    q = QCACHE(Q(index=index, scale=s))
    for sf in softmax:
      for f in filters:
        if sf:            
            qsf = QSOFTMAX(q,dim=[index[0], s])
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print((index, s),sf,f)
            result.append(qsf_f)
        else:
            qsf = QPASS(q,descupdate={'softmax':''})
            qsf_f = QFILTER(qsf,filters[f],filtername=f)
            print(s,sf,f)
            result.append(qsf_f)
  return result

def dict_pos_neg(pos, neg, w):
  return dict(dict_same_weight(1.0*w/len(pos),pos), **dict_same_weight(-1.0*w/len(neg),neg))

def print_permutations(q):
    W = q._pdf['W']
    print(q._descriptor)
    for i, (kmap, w) in enumerate(zip(q._keywords_map, W)):
        context = q._context_template.format_map(kmap)
        answer = q._answer_template.format_map(kmap)
        print(f'{i}.',context ,'->', answer, w)


frequency_weights:SCALE = {
    'never':-4,
    'very rarely':-3,
    'seldom':-2,
    'rarely':-2,
    'frequently':2,
    'often':2,
    'very frequently':3,
    'always':4,    
}
    
binary_frequency_weights = {k: 1 if v > 0 else -1 for k, v in frequency_weights.items()}
rbinary_frequency_weights = {k: -1 if v > 0 else 1 for k, v in frequency_weights.items()}
frequency_pos = [k for k, v in frequency_weights.items() if v > 0]
frequency_neg = [k for k, v in frequency_weights.items() if v < 0]

In [ ]:
p = "typeform/distilbert-base-uncased-mnli"
mnli = pipeline("zero-shot-classification",device=device, model=p)
mnli.model_identifier = p

# ASI

In [ ]:
import qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli
from qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli import asi_qmnli

In [ ]:
asi_questions_equalities = []

for Q in tqdm(asi_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.social_biases.ambivalent_sexism_inventory.asi_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    asi_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in asi_questions_equalities
print("All questions are good - " + str(all_good))
print(asi_questions_equalities)

# BIG5

In [ ]:
import qpsychometric.personality_traits.big5.big5_qmnli
from qpsychometric.personality_traits.big5.big5_qmnli import *

In [ ]:
big5_questions_equalities = []

for Q in tqdm(big5_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.personality_traits.big5.big5_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    big5_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in big5_questions_equalities
print("All questions are good - " + str(all_good))
print(big5_questions_equalities)

# SD3

In [ ]:
import qpsychometric.personality_traits.short_dark_triad.sd3_qmnli
from qpsychometric.personality_traits.short_dark_triad.sd3_qmnli import *

In [ ]:
sd3_questions_equalities = []

for Q in tqdm(sd3_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.personality_traits.short_dark_triad.sd3_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    sd3_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in sd3_questions_equalities
print("All questions are good - " + str(all_good))
print(sd3_questions_equalities)

# SOC

In [ ]:
import qpsychometric.mental_health.sense_of_coherence.soc_qmnli
from qpsychometric.mental_health.sense_of_coherence.soc_qmnli import *

In [ ]:
soc_questions_equalities = []

for Q in tqdm(soc_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.mental_health.sense_of_coherence.soc_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    soc_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in soc_questions_equalities
print("All questions are good - " + str(all_good))
print(soc_questions_equalities)

# PHQ

In [ ]:
import qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli
from qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli import *

In [ ]:
phq_questions_equalities = []

for Q in tqdm(phq_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.mental_health.patient_health_questionnaire.phq_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    phq_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in phq_questions_equalities
print("All questions are good - " + str(all_good))
print(phq_questions_equalities)

# GAD

In [ ]:
import qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli
from qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli import *

In [ ]:
gad_questions_equalities = []

for Q in tqdm(gad_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.mental_health.generalized_anxiety_disorder.gad_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    gad_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in gad_questions_equalities
print("All questions are good - " + str(all_good))
print(gad_questions_equalities)

# Compassion Scale

In [ ]:
import qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli
from qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli import *

In [ ]:
compassion_scale_questions_equalities = []

for Q in tqdm(compassion_scale_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    q_pdf = Qs[0]._pdf
    Q_same = Q.__name__
    Q_func = getattr(qpsychometric.personality_traits.compassion_scale.compassion_scale_qmnli, Q_same)
    Qs = split_question(Q_func,
                        index=Q_func.index,
                        scales=[Q_func.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)
    compassion_scale_questions_equalities.append(q_pdf.equals(Qs[0]._pdf))

all_good = False not in compassion_scale_questions_equalities
print("All questions are good - " + str(all_good))
print(compassion_scale_questions_equalities)

# Load all questions

In [ ]:
import sys, importlib
import qpsychometric
importlib.reload(sys.modules['qpsychometric'])
from qpsychometric.utils import load_questions

In [ ]:
all_qmnli = load_questions()

In [ ]:
all_questions_equalities = asi_questions_equalities + big5_questions_equalities + sd3_questions_equalities + soc_questions_equalities + phq_questions_equalities + gad_questions_equalities + compassion_scale_questions_equalities

for Q in tqdm(all_qmnli):
    Qs = split_question(Q,
                        index=Q.index,
                        scales=[Q.scale],
                        softmax=[True],
                        filters={'unfiltered':{},
                                "positiveonly":Q().get_filter_for_postive_keywords()
                                },
                        )
    print(Qs[0]._descriptor['Ordinal'])
    Qs[0].run(mnli)

all_good = False not in all_questions_equalities
print("All questions are good - " + str(all_good))
print(all_questions_equalities)